In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from dataset import make_dataset
from os.path import join


train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')

x_train, y_train, test = make_dataset(train_path, test_path)

Start time:  2022-10-21 19:02:01.254253
Train dataset success !
Test dataset success !
End time: 2022-10-21 19:03:14.690836
Play time:  0:01:13.436583


In [2]:
import pandas as pd
X = x_train.copy()
y = y_train.copy()
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

In [3]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


param = {'n_estimators': 3516, 'max_depth': 16, 'min_child_weight': 65, 'gamma': 2, 'learning_rate': 0.016, 'colsample_bytree': 0.6669214248431368, 'lambda': 2.4870659647843234, 'alpha': 0.37341889508206516, 'subsample': 0.7}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      XGB = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0)
      XGB.fit(x_train, y_train)
      
      y_pred = XGB.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      XGB_model[f] = XGB
      print(f'================================================================================\n\n')
              


for fold in range(5):
    sample_submission['target'] += XGB_model[fold].predict(test)/5    

c:\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
1 Fold MAE = 3.0283268018311014


====================================2============================================
2 Fold MAE = 3.0270055000116165


====================================3============================================
3 Fold MAE = 3.0239337196073888


====================================4============================================
4 Fold MAE = 3.0238305812440927


====================================5============================================
5 Fold MAE = 3.022662355461429




In [4]:
sample_submission.to_csv("./submit_xgb_fold.csv", index=False)

In [9]:
xgb = pd.read_csv('submit_xgb_fold.csv')
cat = pd.read_csv('submit_cat_kfold.csv')

In [10]:
pred = (xgb['target']*0.5) + (cat['target']*0.5)
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = pred
sample_submission.to_csv("./submit_fold_sum.csv", index=False)